In [30]:
# Imports
import numpy as np

In [31]:
# Simulate passengers function,
def simulate_passengers(mean_arrival_rate, last_minute_ticket_sales=False):
    """
    Simulate the number of passengers who show up for a flight using a Poisson distribution,
    and include last-minute cancellations and ticket sales.

    Args:
        mean_arrival_rate (float): The mean arrival rate for the Poisson distribution.
        last_minute_ticket_sales (bool): Whether last-minute ticket sales are allowed.

    Returns:
        int: The number of passengers who show up for the flight.
    """
    if mean_arrival_rate <= 0:
        return 0
    
    num_passengers = np.random.poisson(mean_arrival_rate)
    SeatCap = 180
    
    if last_minute_ticket_sales:
        num_cancellations = np.random.binomial(num_passengers, 0.05) # Assume 5% chance of cancellation
        num_passengers = max(num_passengers - num_cancellations, 0)
        num_last_minute_sales = SeatCap - num_passengers
        num_last_minute_sales = max(num_last_minute_sales, 0)
        num_passengers += num_last_minute_sales
    
    return num_passengers



In [32]:
def calculate_profit(num_excess_tickets, num_passengers, last_minute_ticket_sales=False):
    """
    Calculate the profit generated by the airline given the number of excess tickets sold and the number of passengers who show up for the flight.

    Args:
    num_excess_tickets (int): The number of excess tickets sold.
    num_passengers (int): The number of passengers who show up for the flight.
    last_minute_ticket_sales (bool): Whether or not to include last minute ticket sales in the calculation.

    Returns:
    float: The profit generated by the airline.
    """
    ticket_price = 200
    overbooking_penalty = 1.25 * ticket_price
    num_paying_passengers = max(num_passengers - num_excess_tickets, 0)

    # Calculate revenue from paying passengers
    if last_minute_ticket_sales:
        num_last_minute_passengers = simulate_passengers(10)
        revenue = (num_paying_passengers + num_last_minute_passengers) * ticket_price
    else:
        revenue = num_paying_passengers * ticket_price

    # Calculate overbooking costs
    overbooking_cost = max(num_excess_tickets - num_passengers, 0) * overbooking_penalty

    # Calculate profit
    profit = revenue - overbooking_cost
    return profit


In [33]:
def monte_carlo_simulation(num_simulations, mean_arrival_rate, excess_tickets_min, excess_tickets_max, last_minute_ticket_sales=False):
    """
    Run a Monte Carlo simulation with a specified number of simulations and optimize over a range of excess tickets sold to find the number that maximizes the average profit generated.

    Args:
    num_simulations (int): The number of simulations to run.
    mean_arrival_rate (float): The mean arrival rate for the Poisson distribution.
    excess_tickets_min (int): The minimum number of excess tickets to sell.
    excess_tickets_max (int): The maximum number of excess tickets to sell.
    last_minute_ticket_sales (bool): Whether last-minute ticket sales are allowed.

    Returns:
    Tuple[float, int]: The average profit generated and the optimal number of excess tickets to sell.
    """
    profits = []
    excess_tickets_list = range(excess_tickets_min, excess_tickets_max + 1)
    for num_excess_tickets in excess_tickets_list:
        total_profit = 0
        for i in range(num_simulations):
            num_passengers = simulate_passengers(mean_arrival_rate, last_minute_ticket_sales)
            total_profit += calculate_profit(num_excess_tickets, num_passengers, last_minute_ticket_sales)
        avg_profit = total_profit / num_simulations
        profits.append(avg_profit)
    optimal_num_excess_tickets = excess_tickets_list[np.argmax(profits)]
    return max(profits), optimal_num_excess_tickets


In [34]:
# Function to present results
def present_results(avg_profit, optimal_num_excess_tickets):
    """
    Present the results of the Monte Carlo simulation to the boss, including the optimal number of excess tickets to sell and the average profit generated.

    Args:
    avg_profit (float): The average profit generated by the airline.
    optimal_num_excess_tickets (int): The optimal number of excess tickets to sell.
    """
    print(f"Optimal number of excess tickets to sell: {optimal_num_excess_tickets}")
    print(f"Average profit generated: ${avg_profit:.2f}")

In [35]:
# testing the code without lst minute cancellations
avg_profit, optimal_num_excess_tickets = monte_carlo_simulation(num_simulations=1000, mean_arrival_rate=170, excess_tickets_min=0, excess_tickets_max=50)
present_results(avg_profit, optimal_num_excess_tickets)

Optimal number of excess tickets to sell: 0
Average profit generated: $34060.40


In [36]:
# testing the code with last minute cancellations
avg_profit, optimal_num_excess_tickets = monte_carlo_simulation(num_simulations=1000, mean_arrival_rate=170, excess_tickets_min=0, excess_tickets_max=50, last_minute_ticket_sales=True)
present_results(avg_profit, optimal_num_excess_tickets)


Optimal number of excess tickets to sell: 0
Average profit generated: $38076.40
